In [30]:
import requests
import pandas as pd
import datetime
import numpy as np
import itertools
import os
import pickle
from sklearn.externals import joblib


In [7]:
accident_dataset = pd.read_csv("C:/Users/sablo/original/CarAccidentPrediction/MyProject/model_Data.csv")

In [9]:
import json
data = {}
data['longitude'] = "-82.74"
data['latitude'] = "27.87"
data['date'] = "02-08-2019 07:00"
data['temp'] = "62.1"
data['pres'] = "30.09"
data['humi'] = "93"
data['visibility'] = "10"
data['windspeed'] = "4.6"
data['weatherkeyword'] = 0
data['roadfeature'] = "Traffic_Signal"
json_data = json.dumps(data)
json_data

'{"longitude": "-82.74", "latitude": "27.87", "date": "02-08-2019 07:00", "temp": "62.1", "pres": "30.09", "humi": "93", "visibility": "10", "windspeed": "4.6", "weatherkeyword": 0, "roadfeature": "Traffic_Signal"}'

In [11]:
import requests

url = 'http://127.0.0.1:5000/predict'
headers = {'content-type': 'application/json'}
r = requests.post(url, verify=False, data=json_data, headers=headers)


<Response [500]>

In [44]:
def find_cluster(accident_dataset, lat, long):
    # load all cluster accident waypoints to check against proximity
    accident_point_counts = len(accident_dataset.index)
    cluster = 1
    # approximate radius of earth in km
    R = 6373.0
    for index, row in accident_dataset.iterrows():
        lat = np.radians(lat)
        lng = np.radians(long)
        latref = row['Start_Lat']
        lngref = row['Start_Lng']
        dlat = lat - latref
        dlng = lng - lngref
        a = np.sin(dlat / 2) ** 2 + np.cos(latref) * np.cos(lat) * np.sin(dlng/ 2) ** 2
        dist = R * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
        
        if (dist < 0.10):
            cluster = row['Cluster']
            break


    return cluster

In [41]:
def create_model_input(request):
    inputdata = pd.Series(np.zeros(24), index = ['Start_Lat', 'Start_Lng', 'Cluster', 'Weekday', 'Hour', 'Date', 'Month',
       'Sunrise_Sunset', 'Traffic_Signal', 'Station', 'Junction', 'Crossing',
       'Temperature(F)', 'Wind_Speed(mph)', 'Pressure(in)', 'Visibility(mi)',
       'Humidity(%)', 'cloudy', 'fog', 'overcast', 'rain', 'snow',
       'thunderstorm', 'wind'])
    inputdata['Cluster'] = find_cluster(accident_dataset,float(request['latitude']),float(request['longitude']))
    inputdata['Start_Lat'] = float(request['latitude'])
    inputdata['Start_Lng'] = float(request['longitude'])
    inputdata['Temperature(F)'] = float(request['temp'])
    inputdata['Pressure(in)'] = float(request['pres'])
    inputdata['Wind_Speed(mph)'] = float(request['windspeed'])
    inputdata['Humidity(%)'] = float(request['humi'])
    inputdata['Visibility(mi)'] = float(request['visibility'])
    inputdata[request['roadfeature']] = 1
    if (request['weatherkeyword'] != 0):
        inputdata[request['weatherkeyword']] = 1
    
    datetime_object = datetime.datetime.strptime(request['date'], '%d-%m-%Y %H:%M')
    inputdata['Hour'] = datetime_object.hour
    inputdata['Date'] = datetime_object.day
    inputdata['Month'] = datetime_object.month
    inputdata['Weekday'] = datetime_object.date().weekday()
    inputdata['Sunrise_Sunset'] = 1 if datetime_object.hour < 18 and datetime_object.hour > 6 else 0
    
    return inputdata
    

In [45]:
modeinput = create_model_input(data)

In [31]:
filename = 'C:/Users/sablo/original/CarAccidentPrediction/MyProject/FlaskAPI/accident-prediction-rfc-model.pkl'
classifier = pickle.load(open(filename, 'rb'))

In [36]:
my_prediction = classifier.predict(modeinput.values.reshape(1,-1))[0]
my_prediction

0

In [47]:
modeinput

Start_Lat          27.87
Start_Lng         -82.74
Cluster             1.00
Weekday             4.00
Hour                7.00
Date                2.00
Month               8.00
Sunrise_Sunset      1.00
Traffic_Signal      1.00
Station             0.00
Junction            0.00
Crossing            0.00
Temperature(F)     62.10
Wind_Speed(mph)     4.60
Pressure(in)       30.09
Visibility(mi)     10.00
Humidity(%)        93.00
cloudy              0.00
fog                 0.00
overcast            0.00
rain                0.00
snow                0.00
thunderstorm        0.00
wind                0.00
dtype: float64